In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Basics').getOrCreate()

23/05/01 11:07:49 WARN Utils: Your hostname, mia-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/05/01 11:07:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/01 11:07:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/01 11:07:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/01 11:07:50 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/05/01 11:07:50 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
resource_url = '/home/mia/Downloads/Python-and-Spark-for-Big-Data-master/'
df = spark.read.csv(
    resource_url + 'Spark_DataFrames/' + 'ContainsNull.csv',
    inferSchema=True,
    header=True
)

In [4]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [3]:
df.toPandas()

,Id,Name,Sales
0,emp1,John,NaN
1,emp2,None,NaN
2,emp3,None,345.0
3,emp4,Cindy,456.0


In [5]:
df.na.drop().toPandas()

,Id,Name,Sales
0,emp4,Cindy,456.0


In [6]:
df.na.drop(thresh=2).toPandas() # less than thresh, non-null values

,Id,Name,Sales
0,emp1,John,NaN
1,emp3,None,345.0
2,emp4,Cindy,456.0


In [7]:
df.na.drop(how='any').toPandas()

,Id,Name,Sales
0,emp4,Cindy,456.0


In [8]:
df.na.drop(how='all').toPandas()

,Id,Name,Sales
0,emp1,John,NaN
1,emp2,None,NaN
2,emp3,None,345.0
3,emp4,Cindy,456.0


In [9]:
df.na.drop(subset=['Sales']).toPandas()

,Id,Name,Sales
0,emp3,None,345.0
1,emp4,Cindy,456.0


In [10]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [12]:
df.na.fill('FILL VALUE').toPandas()

,Id,Name,Sales
0,emp1,John,NaN
1,emp2,FILL VALUE,NaN
2,emp3,FILL VALUE,345.0
3,emp4,Cindy,456.0


In [14]:
df.na.fill(0).toPandas()

,Id,Name,Sales
0,emp1,John,0.0
1,emp2,None,0.0
2,emp3,None,345.0
3,emp4,Cindy,456.0


In [15]:
df.na.fill('No Name', subset=['Name']).toPandas()

,Id,Name,Sales
0,emp1,John,NaN
1,emp2,No Name,NaN
2,emp3,No Name,345.0
3,emp4,Cindy,456.0


In [16]:
from pyspark.sql.functions import mean

In [21]:
mean_res = df.select(mean(df['Sales'])).collect()

In [22]:
mean_res

[Row(avg(Sales)=400.5)]

In [24]:
mean_val = mean_res[0][0]
mean_val

400.5

In [25]:
df.na.fill(mean_val, ['Sales']).toPandas()

,Id,Name,Sales
0,emp1,John,400.5
1,emp2,None,400.5
2,emp3,None,345.0
3,emp4,Cindy,456.0


In [26]:
df.na.fill(
    df.select(mean(df['Sales'])).collect()[0][0], 
    ['Sales'])
.toPandas()

,Id,Name,Sales
0,emp1,John,400.5
1,emp2,None,400.5
2,emp3,None,345.0
3,emp4,Cindy,456.0
